In [1]:
from pydriller import Repository
from collections import defaultdict

from pathlib import Path

TARGET_REPO = "devQ_testData_PythonProject"
# put the target repo next to the current folder

# Get the path to the current project (assumes script is run from the project directory)
current_repo_path = Path().parent.resolve()
# print(current_repo_path)

# Go up one level, then into the sibling directory
repo_path = current_repo_path.parent / TARGET_REPO

author_stats = defaultdict(lambda: {"commits": 0, "insertions": 0, "deletions": 0})
author_mapping = defaultdict(lambda: set())

repo = Repository(str(repo_path))

In [217]:
def commits():
    for commit in repo.traverse_commits():
        if not commit.msg.startswith('Merge'):
            continue
        assert len(commit.parents) == 2
        
        yield commit

a = commits()

In [226]:
commit = next(a)
commit.msg, commit.files, commit.modified_files, commit.lines

("Merge remote-tracking branch 'origin/master'", 1, [], 43)

In [227]:
commit.hash

'2d2d8c722044eb830694ced17e55f2bcc55445bd'

'66d418305f8a7b9fa70b2bd880b1d6c26b49e9ff'

In [34]:
commit.branches

{'develop'}

In [33]:
for mod in commit.modified_files:
    print(mod.filename)
    print(mod.diff)

requirements.txt
@@ -36,4 +36,3 @@ typing-extensions==3.7.4.3
 urllib3==1.26.7
 xlrd==2.0.1
 zipp==3.4.1
-celery==5.2.7
\ No newline at end of file

